In [18]:
import pandas as pd
import numpy as np
import datetime 
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [10]:
#import raw dataset
raw_df = pd.read_excel('blu365_dataset_estudodecaso.xlsx')
raw_df

,datadisparo,ddd,sms_entregue,sms_enviado,sms_naoentregue,sms_rejeitado,sms_invalido,data_atraso,valor,operadora,...,data_maisantiga_acesso,sms_total_tent,idade,genero,compradoronline,vendedoronline,utilizacaoweb,rendaestimada,acesso,acordo
0,2020-05-19,62,0,0,0,0,0,2019-05-24,126.21,Claro,...,NaT,0,22.0,M,H,H,H,NaN,0,0
1,2020-04-27,67,3,0,0,0,0,2017-04-14,244.29,Vivo,...,NaT,3,56.0,F,D,H,D,ATE 2 SM,0,0
2,2020-05-14,51,1,0,0,0,0,2019-09-04,51.00,Claro,...,NaT,1,45.0,M,B,H,B,ATE 2 SM,0,0
3,2020-05-13,51,0,0,0,0,0,2019-11-14,307.91,Tim,...,NaT,0,35.0,F,B,H,B,4 A 10 SM,0,0
4,2020-05-25,62,0,0,12,0,0,2017-03-14,154.12,Tim,...,NaT,12,43.0,F,A,H,A,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686378,2020-05-21,51,0,0,0,0,0,2017-03-27,215.34,Tim,...,NaT,0,68.0,M,G,H,G,NaN,0,0
686379,2020-05-07,61,0,0,5,0,0,2018-06-14,62.59,Claro,...,NaT,5,20.0,F,F,H,F,ATE 2 SM,0,0
686380,2020-05-18,41,4,0,0,0,0,2019-11-14,430.02,Tim,...,NaT,4,54.0,F,A,H,A,ATE 2 SM,0,0
686381,2020-05-13,41,0,0,0,0,0,2019-12-14,278.78,Tim,...,NaT,0,39.0,F,B,H,B,ATE 2 SM,0,0


In [29]:
#treating df
le=LabelEncoder()
df=pd.DataFrame()
df["Diferença dias"]=(raw_df['datadisparo']-raw_df['data_atraso']).dt.days
df[['sms_entregue',	'sms_enviado','sms_naoentregue','sms_rejeitado','sms_invalido','valor','qtd_hist_acessos','sms_total_tent','idade']]=raw_df[
    ['sms_entregue','sms_enviado','sms_naoentregue','sms_rejeitado','sms_invalido','valor','qtd_hist_acessos','sms_total_tent','idade']]
df['rendaestimada']=le.fit_transform(raw_df['rendaestimada'].fillna('SEM INFORMACAO'))
df['ddd']=le.fit_transform(raw_df['ddd'])
df['operadora']=le.fit_transform(raw_df['operadora'])
df['genero']=le.fit_transform(raw_df['genero'])
df['compradoronline']=le.fit_transform(raw_df['compradoronline'])
df['vendedoronline']=le.fit_transform(raw_df['vendedoronline'])
df['utilizacaoweb']=le.fit_transform(raw_df['utilizacaoweb'])
df[['acesso','acordo']]=raw_df[['acesso','acordo']]
df.dropna(inplace=True)


df


,Diferença dias,sms_entregue,sms_enviado,sms_naoentregue,sms_rejeitado,sms_invalido,valor,qtd_hist_acessos,sms_total_tent,idade,rendaestimada,ddd,operadora,genero,compradoronline,vendedoronline,utilizacaoweb,acesso,acordo
0,361,0,0,0,0,0,126.21,0,0,22.0,7,35,0,1,7,7,7,0,0
1,1109,3,0,0,0,0,244.29,0,3,56.0,5,40,5,0,3,7,3,0,0
2,253,1,0,0,0,0,51.00,0,1,45.0,5,30,0,1,1,7,1,0,0
3,181,0,0,0,0,0,307.91,0,0,35.0,3,30,4,0,1,7,1,0,0
4,1168,0,0,12,0,0,154.12,0,12,43.0,7,35,4,0,0,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686378,1151,0,0,0,0,0,215.34,0,0,68.0,7,30,4,1,6,7,6,0,0
686379,693,0,0,5,0,0,62.59,0,5,20.0,5,34,0,0,5,7,5,0,0
686380,186,4,0,0,0,0,430.02,0,4,54.0,5,21,4,0,0,7,0,0,0
686381,151,0,0,0,0,0,278.78,0,0,39.0,5,21,4,0,1,7,1,0,0


In [33]:
#creating model
ds=df.values
X=ds[:,0:18].astype(float)


array([[3.610e+02, 0.000e+00, 0.000e+00, ..., 7.000e+00, 0.000e+00,
        0.000e+00],
       [1.109e+03, 3.000e+00, 0.000e+00, ..., 3.000e+00, 0.000e+00,
        0.000e+00],
       [2.530e+02, 1.000e+00, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.860e+02, 4.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.510e+02, 0.000e+00, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [1.085e+03, 0.000e+00, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00]])